#Install Libraries

In [ ]:
!pip install -U datasets transformers accelerate pandas pyarrow chardet nltk spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This 

#Import Libraries

In [ ]:
import re,string
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Load Dataset

In [ ]:
#From huggingface
dataset = load_dataset('imdb')
print(dataset)

#Custom
#dataset = load_dataset("csv",data_files={'data':'custom.csv'})['data']
#print(dataset)

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


#Clean Data

In [ ]:
re_url=True
re_html=True
re_stopwords=False
lower=True
re_punc=True

urls = re.compile(r"(www\.\S+|https?://\S+)")
htmls = re.compile(r"(<.*?>)")
stopwords = set(stopwords.words("english")) if re_stopwords else set()
punct = str.maketrans("","",string.punctuation)

In [ ]:
def clean_text(x):
  if re_url:
    x=urls.sub(" ",x)
  if re_html:
    x=htmls.sub(" ",x)
  if lower:
    x=x.lower()
  if re_stopwords:
    x= " ".join([w for w in x.split() if w not in stopwords])
  if re_punc:
    x=x.translate(punct)
  return x

In [ ]:
def apply_clean(batch):
  return {"text":[clean_text(x) for x in batch["text"]]}

In [ ]:
dataset=dataset.map(apply_clean,batched=True)
print(dataset["train"][0])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': 'I rented I AM CURIOUSYELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967 I also heard that at first it was seized by US customs if it ever tried to enter this country therefore being a fan of films considered controversial I really had to see this for myself  The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States In between asking politicians and ordinary denizens of Stockholm about their opinions on politics she has sex with her drama teacher classmates and married men  What kills me about I AM CURIOUSYELLOW is that 40 years ago this was considered pornographic Really the sex and nudity scenes are few and far between even then its not shot like some

#Tokenize

In [ ]:
model_name='bert-base-uncased'
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
max_len = 256

def tokenize(batch):
  return tokenizer(batch['text'], truncation=True, max_length=max_len)

In [ ]:
tokenized = dataset.map(tokenize, batched=True, remove_columns=["text"])
print(tokenized["train"][0])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'label': 0, 'input_ids': [101, 1045, 12524, 1045, 2572, 8025, 6672, 7174, 2860, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 2149, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 6801, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 2055, 2037, 10740, 2006, 4331, 2016, 2038, 3348, 2007, 2014, 3689, 3836, 19846, 1998, 2496, 2273, 2054, 8563, 2033, 2055, 1045, 2572, 8025, 6672, 7174, 2860, 2003, 20

#Padding & Mask

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_loader = DataLoader(tokenized["train"], batch_size=8, collate_fn=data_collator, shuffle=True)

In [ ]:
batch = next(iter(train_loader))
for k, v in batch.items():
    print(k, v.shape)

input_ids torch.Size([8, 256])
token_type_ids torch.Size([8, 256])
attention_mask torch.Size([8, 256])
labels torch.Size([8])


In [ ]:
tokenized.save_to_disk("processed_dataset")
print("Dataset ready & saved!")

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset ready & saved!
